In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('data_eda.csv')

### Choose Relevant Models

In [ ]:
df.columns

In [ ]:
df_model = df[['avg_salary', 'Rating', 'Size', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'num_comp', 'hourly', 'employer_provided', 'job_state', 'same_state', 'age', 'python', 'aws', 'excel', 'spark', 'excel', 'job_simp', 'seniority', 'desc_len']]

### Get Dummy Data Variables

In [ ]:
df_dum = pd.get_dummies(df_model)

### Train, Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X = df_dum.drop('avg_salary', axis = 1)
y = df_dum.avg_salary.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
list(X_test.iloc[1, :])

### Multi-linear Regression

In [ ]:
import statsmodels.api as sm
X_sm = X =  sm.add_constant(X)
model = sm.OLS(y.astype(float), X.astype(float))
model.fit().summary()

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import cross_val_score

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)

In [ ]:
cross_val_score(lm, X_train, y_train, scoring = 'neg_mean_absolute_error', cv = 3)

### Lasso Regression

In [ ]:
lml = Lasso(alpha = 0.13)
lml.fit(X_train, y_train)

In [ ]:
cross_val_score(lml, X_train, y_train, scoring = 'neg_mean_absolute_error', cv = 3)

In [ ]:
alpha = []
error = []

for i in range(1, 100):
    alpha.append(i/100)
    lml = Lasso(alpha = (i/100))
    error.append(np.mean(cross_val_score(lml, X_train, y_train, scoring = 'neg_mean_absolute_error', cv = 3)))

plt.plot(alpha, error)

In [ ]:
error = tuple(zip(alpha, error))

In [ ]:
df_err = pd.DataFrame(error, columns = ['alpha', 'error'])

In [ ]:
df_err[df_err.error == max(df_err.error)]

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
np.mean(cross_val_score(rf, X_train, y_train, scoring = 'neg_mean_absolute_error', cv = 3))

### Hyperparameter Tuning: GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators': range(10, 300, 10), 'criterion': ("squared_error", "absolute_error"), 'max_features': ('auto', 'sqrt', 'log2')}

In [ ]:
gs = GridSearchCV(rf, parameters, scoring = 'neg_mean_absolute_error', cv = 3)
gs.fit(X_train, y_train)

In [ ]:
gs.best_score_
gs.best_estimator_

### Test Ensembles

In [ ]:
tpred_lm = lm.predict(X_test)
tpred_lml = lml.predict(X_test)
tpred_rf = gs.best_estimator_.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, tpred_lm)

In [ ]:
mean_absolute_error(y_test, tpred_lml)

In [ ]:
mean_absolute_error(y_test, tpred_rf)

In [ ]:
mean_absolute_error(y_test, (tpred_lml + tpred_rf)/2)

In [ ]:
import pickle

In [ ]:
pickl = {'model': gs.best_estimator_}
pickle.dump( pickl, open( 'model_file' + ".p", "wb" ))

In [ ]:
file_name = "model_file.p"
with open(file_name, 'rb') as pickled:
    data = pickle.load(pickled)
    model = data['model']

In [ ]:
model.predict(X_test.iloc[1, :].values.reshape(1, -1))